# ChatGPTで社内チャットボットを作成する方法

In [1]:
#@title 1. Googleドライブに接続
from IPython.display import Markdown, display
from google.colab import drive
import os
drive.mount('/content/drive', force_remount=True)
DRIVE_BASE_DIR = "/content/drive/MyDrive" # Myドライブの場所
os.chdir(DRIVE_BASE_DIR)

Mounted at /content/drive


In [ ]:
#@title 2. 必要なライブラリのインストール
!pip install openai langchain faiss-cpu bs4 libmagic pdfminer.six unstructured[local-inference]

In [37]:
#@title 3. Remixのソースコードを取得
!rm -rf remix
!git clone https://github.com/remix-run/remix
!rm -rf remix/.git

Cloning into 'remix'...
remote: Enumerating objects: 103145, done.
remote: Counting objects: 100% (778/778), done.
remote: Compressing objects: 100% (434/434), done.
remote: Total 103145 (delta 466), reused 562 (delta 344), pack-reused 102367
Receiving objects: 100% (103145/103145), 34.55 MiB | 9.79 MiB/s, done.
Resolving deltas: 100% (88308/88308), done.
Updating files: 100% (903/903), done.


In [ ]:
#@title 4. テキストファイルから学習する
import pickle

from langchain.document_loaders import DirectoryLoader, TextLoader, ReadTheDocsLoader
from langchain.document_loaders.pdf import (
    PDFMinerLoader,
    PyMuPDFLoader,
    UnstructuredPDFLoader,
)
from langchain.document_loaders.html import UnstructuredHTMLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
import os

openai_api_key= "\u003COPENAI API KEY>" #@param {'type': 'string'}
os.environ["OPENAI_API_KEY"] = openai_api_key
vectorstore_file= "vectorstore" #@param {'type': 'string'}


def ingest_docs(dir_name):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=200,
    )
    raw_documents = []
    for loader_cls in [TextLoader, PDFMinerLoader]:
        loader = DirectoryLoader(dir_name, loader_cls=loader_cls, silent_errors=True)
        raw_documents += loader.load()
    documents = text_splitter.split_documents(raw_documents)
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(documents, embeddings)

    # Save vectorstore
    with open(f"{vectorstore_file}.pkl", "wb") as f:
        pickle.dump(vectorstore, f)

dir_name = "remix" #@param {'type': 'string'}
ingest_docs(dir_name)

In [45]:
#@title 5. モデルの読み込みとLLMChainの準備
from langchain.chains.llm import LLMChain
from langchain.callbacks.base import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.openai_info import OpenAICallbackHandler
from langchain.chains.chat_vector_db.prompts import CONDENSE_QUESTION_PROMPT, QA_PROMPT
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAIChat
from langchain.vectorstores.base import VectorStore
from langchain.chains import ChatVectorDBChain

vectorstore_file= "vectorstore" #@param {'type': 'string'}
with open(f"{vectorstore_file}.pkl", "rb") as f:
    vectorstore = pickle.load(f)

manager = CallbackManager([StreamingStdOutCallbackHandler(), OpenAICallbackHandler()])

streaming_llm = OpenAIChat(streaming=True, callback_manager=manager, verbose=True, temperature=0)
question_gen_llm = OpenAIChat(temperature=0, verbose=True, callback_manager=manager)

question_generator = LLMChain(llm=question_gen_llm, prompt=CONDENSE_QUESTION_PROMPT)
doc_chain = load_qa_chain(streaming_llm, chain_type="stuff", prompt=QA_PROMPT)

chat_history = []
qa = ChatVectorDBChain(vectorstore=vectorstore, combine_docs_chain=doc_chain, question_generator=question_generator)

### 6. 質問してみる

In [46]:
chat_history = []
question = "remixは既存のフレームワークと何が違うのですか？日本語で箇条書きにしてくださ"
display(Markdown(f"# 1. [質問] \n## {question}"))
display(Markdown("# 2. [回答]"))
result = qa({"question": question, "chat_history": chat_history})
#chat_history.append((question, result["answer"]))

display(Markdown("# 3. [類似のコンテキスト]"))
for context in vectorstore.similarity_search(question):
    display(Markdown(context.page_content))

# 1. [質問] 
## remixは既存のフレームワークと何が違うのですか？日本語で箇条書きにしてくださ

# 2. [回答]

- Remixは、標準的なAPIを使用しやすくすることを目的としています。
- Remixは、Web開発全般について学ぶことができます。
- Remixは、ViewとControllerの役割を担うフレームワークです。
- Remixは、モデルについてはユーザーに任せています。
- Remixは、Fetch APIに関するヘルパーを提供しています。
- Remixは、Node.jsサーバーおよびCloudflare Workersなどの非Node.js環境にデプロイできます。

# 3. [類似のコンテキスト]

Remix's job is to cross the center of the stack and then get out of your way. We avoid as many "Remixisms" as possible and instead make it easier to use the standard APIs the web already has.

This one is more for us. We've been educators for the 5 years before Remix. Our tagline is _Build Better Websites_. We also think of it with a little extra on the end: _Build Better Websites, Sometimes with Remix_. If you get good at Remix, you will accidentally get good at web development in general.

Remix's APIs make it convenient to use the fundamental Browser/HTTP/JavaScript, but those technologies are not hidden from you.

[git-hub]: https://github.com/remix-run/remix

Additionally, if Remix doesn't have an adapter for your server already, you can look at the source of one of the adapters and build your own.

## Server Framework

If you're familiar with server-side MVC web frameworks like Rails and Laravel, Remix is the View and Controller, but it leaves the Model up to you. There are a lot of great databases, ORMs, mailers, etc. in the JavaScript ecosystem to fill that space. Remix also has helpers around the Fetch API for cookie and session management.

Instead of having a split between View and Controller, Remix Route modules take on both responsibilities.

Most server-side frameworks are "model focused". A controller manages _multiple URLs_ for a single model.

# Welcome to Remix!

We are happy you're here!

[Remix](https://remix.run) is a full stack web framework that lets you focus on the user interface and work back through web fundamentals to deliver a fast, slick, and resilient user experience that deploys to any Node.js server and even non-Node.js environments at the edge like Cloudflare Workers.

Want to know more? Read the [Technical Explanation of Remix](https://remix.run/pages/technical-explanation)

This repository contains the Remix source code. This repo is a work in progress, so we appreciate your patience as we figure things out.

## Documentation

For documentation about Remix, please visit [our website](https://remix.run/docs).

Also, please [join our community on Discord](https://rmx.as/discord).

In [43]:
chat_history = []
question = "remixのディレクトリ階層を教えてください。"
display(Markdown(f"# 1. [質問] \n## {question}"))
display(Markdown("# 2. [回答]"))
result = qa({"question": question, "chat_history": chat_history})
#chat_history.append((question, result["answer"]))

display(Markdown("# 3. [類似のコンテキスト]"))
for context in vectorstore.similarity_search(question):
    display(Markdown(context.page_content))

# 1. [質問] 
## remixのディレクトリ階層を教えてください。

# 2. [回答]

remix-jokes
├── README.md
├── app
│   ├── entry.client.tsx
│   ├── entry.server.tsx
│   ├── root.tsx
│   └── routes
│       └── index.tsx
├── package-lock.json
├── package.json
├── public
│   └── favicon.ico
├── remix.config.js
├── remix.env.d.ts
└── tsconfig.json

# 3. [類似のコンテキスト]

[git-hub]: https://github.com/remix-run/remix

Once the `npm install` has completed, we'll change into the `remix-jokes` directory:

💿 Run this command

```sh
cd remix-jokes
```

Now you're in the `remix-jokes` directory. All other commands you run from here on out will be in that directory.

💿 Great, now open that up in your favorite editor and let's explore the project structure a bit.

## Explore the project structure

Here's the tree structure. Hopefully what you've got looks a bit like this:

```
remix-jokes
├── README.md
├── app
│   ├── entry.client.tsx
│   ├── entry.server.tsx
│   ├── root.tsx
│   └── routes
│       └── index.tsx
├── package-lock.json
├── package.json
├── public
│   └── favicon.ico
├── remix.config.js
├── remix.env.d.ts
└── tsconfig.json
```

Let's talk briefly about a few of these files:

Remix examples have moved into a dedicated repository. [See `remix-run/examples`](https://github.com/remix-run/examples) to browse or contribute new example projects!

Remix's job is to cross the center of the stack and then get out of your way. We avoid as many "Remixisms" as possible and instead make it easier to use the standard APIs the web already has.

This one is more for us. We've been educators for the 5 years before Remix. Our tagline is _Build Better Websites_. We also think of it with a little extra on the end: _Build Better Websites, Sometimes with Remix_. If you get good at Remix, you will accidentally get good at web development in general.

Remix's APIs make it convenient to use the fundamental Browser/HTTP/JavaScript, but those technologies are not hidden from you.

In [55]:
chat_history = []
question = "root.tsxのサンプルを示してください"
display(Markdown(f"# 1. [質問] \n## {question}"))
display(Markdown("# 2. [回答]"))
result = qa({"question": question, "chat_history": chat_history})
#chat_history.append((question, result["answer"]))

display(Markdown("# 3. [類似のコンテキスト]"))
for context in vectorstore.similarity_search(question):
    display(Markdown(context.page_content))

# 1. [質問] 
## root.tsxのサンプルを示してください

# 2. [回答]

The sample code for `root.jsx` is already provided in the given context. Here it is again:

```jsx
import { Outlet } from "@remix-run/react";

export default function Root() {
  return (
    <html lang="en">
      <head>
        <meta charSet="utf-8" />
        <link rel="icon" href="/favicon.ico" />
        <meta
          name="viewport"
          content="width=device-width, initial-scale=1"
        />
        <meta name="theme-color" content="#000000" />
        <meta
          name="description"
          content="My beautiful React app"
        />
        <link rel="apple-touch-icon" href="/logo192.png" />
        <link rel="manifest" href="/manifest.json" />
        <title>My React App</title>
      </head>
      <body>
        <div id="root">
          <Outlet />
        </div>
      </body>
    </html>
  );
}
```

# 3. [類似のコンテキスト]

```js filename=root.jsx
import { Outlet } from "@remix-run/react";

export default function Root() {
  return (
    <html lang="en">
      <head>
        <meta charSet="utf-8" />
        <link rel="icon" href="/favicon.ico" />
        <meta
          name="viewport"
          content="width=device-width, initial-scale=1"
        />
        <meta name="theme-color" content="#000000" />
        <meta
          name="description"
          content="My beautiful React app"
        />
        <link rel="apple-touch-icon" href="/logo192.png" />
        <link rel="manifest" href="/manifest.json" />
        <title>My React App</title>
      </head>
      <body>
        <div id="root">
          <Outlet />
        </div>
      </body>
    </html>
  );
}
```

Notice a few things here:

export default function Root() {
              return (
                <html>
                  <body>
                    ${HOME_PAGE_TEXT}
                    <Outlet />
                  </body>
                </html>
              );
            }
          `,
          "app/routes/redirect.jsx": js`
            import { redirect } from "@remix-run/node";
            export const loader = () => redirect("/redirect-target");
            export default () => <div>Yo</div>
          `,
          "app/routes/redirect-target.jsx": js`
            export default () => <div>${REDIRECT_TARGET_TEXT}</div>
          `,
          "app/routes/fetch.jsx": js`
            export function loader({ request }) {
              return fetch(new URL(request.url).origin + '/fetch-target');
            }

```jsx filename=root.jsx lines=[1,4,6-15,31]
import { Links } from "@remix-run/react";

import App from "./app";
import stylesheetUrl from "./styles.css";

export function links() {
  // `links` returns an array of objects whose
  // properties map to the `<link />` component props
  return [
    { rel: "icon", href: "/favicon.ico" },
    { rel: "apple-touch-icon", href: "/logo192.png" },
    { rel: "manifest", href: "/manifest.json" },
    { rel: "stylesheet", href: stylesheetUrl },
  ];
}

The root route (or the "root root" if you're Wes Bos) is responsible for providing the structure of the application. Its default export is a component that renders the full HTML tree that every other route loads and depends on. Think of it as the scaffold or shell of your app.

In a client-rendered app, you will have an index HTML file that includes the DOM node for mounting your React app. The root route will render markup that mirrors the structure of this file.

Create a new file called `root.jsx` (or `root.tsx`) in your `app` directory. The contents of that file will vary, but let's assume that your `index.html` looks something like this: